# RNN models
> Various RNNs and dynamical models.

In [ ]:
#| default_exp recurrent.models

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from wafer.basics import *

## Utils

In [ ]:
#| export
class Select(nn.Module):
    "Select from iterable."
    def __init__(self, idx=0):
        super().__init__()
        self.idx = idx
    def forward(self, x):
        return x[self.idx]

## Basic

In [ ]:
#| export
from wafer.init import default_init, lambda_init

In [ ]:
#| export
class SimpleRNN(nn.Module):
    "A RNN with its output mapped through a dense layer."
    def __init__(self, ni, nh, no, num_layers=2,
                 actn='tanh',   # nonlinearity, ['tanh', 'relu']
                 init='normal', # initialization method ['uniform', 'normal', 'irnn', 'np-rnn']
                ):
        super().__init__()
        self.recurrent = nn.RNN(ni, nh, num_layers=num_layers, batch_first=True, nonlinearity=actn)
        self.dense = nn.Linear(nh, no)
        # Initialize
        self._init(self.recurrent, nh, init)
        default_init(self.dense)

    @staticmethod
    @torch.no_grad()
    def _init(m, nh, init):
        for n,p in m.named_parameters():
            if 'bias' in n:
                nn.init.zeros_(p)
            if 'weight_ih' in n:
                if init == 'uniform':
                    nn.init.uniform_(p, a=-1/np.sqrt(nh), b=1/np.sqrt(nh))
                elif init == 'np-rnn':
                    nn.init.normal_(p, std=1/np.sqrt(nh))
                    p = p * (np.sqrt(2) * np.exp(1.2 / (max(nh, 6) - 2.4)))
                    getattr(m, n).copy_(p)
                else:
                    nn.init.normal_(p, std=1/np.sqrt(nh))
            if 'weight_hh' in n:
                if init == 'uniform':
                    nn.init.uniform_(p, a=-1/np.sqrt(nh), b=1/np.sqrt(nh))
                elif init == 'irnn':
                    nn.init.eye_(p)
                elif init == 'np-rnn':
                    nn.init.normal_(p)
                    p = (p.T @ p) / nh
                    p = p / np.linalg.eigvals(p.detach().numpy()).max()
                    getattr(m, n).copy_(p)
                else:
                    nn.init.normal_(p, std=1/np.sqrt(nh))
    
    def forward(self, x):
        outp,_ = self.recurrent(x) # shape(N, L, nh)
        return self.dense(outp)

In [ ]:
#| export
class LSTM(nn.Module):
    """Custom LSTM, allowing for different activations. Assuming `batch_first=True` and `bidirectional=False`.
    
    Inputs: x, h0_c0;
        x: shape (N, L, input_size)
        h0_c0: optional, default zeros, shape (num_layers, N, hidden_size)
    
    Outputs: output, (hn, cn);
        output: shape (N, L, hidden_size), outputs of the last layer for each token.
        hn: shape (N, num_layers, hidden_size), final hidden state.
        cn: shape (N, num_layers, hidden_size), final cell state.
    
    """
    def __init__(self, ni, nh, num_layers=1, actn='tanh', gate_actn='sigmoid',
                 dropout=0.0, unit_forget_bias=True, init_gain=1/np.sqrt(3), recurrent_init_gain=1.):
        super().__init__()
        self.nh,self.num_layers = nh,num_layers
        self.actn = getattr(F, actn, 'tanh')
        self.gate_actn = getattr(F, gate_actn, 'sigmoid')
        self.dropout = nn.Dropout(p=dropout)

        ws = []
        for i in range(num_layers):
            ws.append(nn.ModuleDict({
                'ii': nn.Linear(ni if i == 0 else nh, nh),  # input gate
                'hi': nn.Linear(nh, nh),
                'if': nn.Linear(ni if i == 0 else nh, nh),  # forget gate
                'hf': nn.Linear(nh, nh),
                'io': nn.Linear(ni if i == 0 else nh, nh),  # output gate
                'ho': nn.Linear(nh, nh),
                'ic': nn.Linear(ni if i == 0 else nh, nh),  # cell gate
                'hc': nn.Linear(nh, nh)
            }))
        self.ws = nn.ModuleList(ws)
        
        # Initialize
        for n,p in self.named_parameters():
            # hidden/recurrent weight
            if '.h' in n:
                if 'bias' in n:
                    nn.init.zeros_(p)
                else:
                    nn.init.orthogonal_(p, recurrent_init_gain)
            # non-recurrent weight
            else:
                if 'bias' in n:
                    if '.if' in n and unit_forget_bias:
                        nn.init.ones_(p)
                    else:
                        nn.init.zeros_(p)
                else:
                    nn.init.xavier_uniform_(p, init_gain) 
    
    def _forward_single(self, x, h0_c0: list=None):
        "Forward pass of a single token."
        if h0_c0 is None:
            h0 = torch.zeros(self.num_layers, x.shape[0], self.nh, device=x.device)
            c0 = torch.zeros(self.num_layers, x.shape[0], self.nh, device=x.device)
        else:
            h0,c0 = h0_c0
            assert (h0.shape[-1] == c0.shape[-1] == self.nh) and (h0.shape[1] == c0.shape[1] == x.shape[0])
        
        hs, cs = [], []
        for i in range(self.num_layers):
            h, c = h0[i], c0[i]
            i_gate = self.ws[i]['ii'](x) + self.ws[i]['hi'](h)
            f_gate = self.ws[i]['if'](x) + self.ws[i]['hf'](h)
            o_gate = self.ws[i]['io'](x) + self.ws[i]['ho'](h)
            c_gate = self.ws[i]['ic'](x) + self.ws[i]['hc'](h)

            i_gate = self.gate_actn(i_gate)
            f_gate = self.gate_actn(f_gate)
            o_gate = self.gate_actn(o_gate)
            c_gate = self.actn(c_gate)

            c_new = (f_gate * c) + (i_gate * c_gate)
            h_new = o_gate * self.actn(c_new)
            cs.append(c_new)
            hs.append(h_new)
            x = self.dropout(h_new)

        hs, cs = torch.stack(hs, 0), torch.stack(cs, 0)
        return (hs, cs)

    def forward(self, x, h0_c0=None):
        # Input `x`, shape (N,L,D); `h0_c0`, shape (num_layers, N, nh).
        hs = []
        cs = []
        for xi in torch.permute(x, [1,0,2]):
            hc = self._forward_single(xi, h0_c0)
            hs.append(hc[0])
            cs.append(hc[1])
            h0_c0 = hc
        output = torch.stack([h[-1] for h in hs], 1)
        return output, (hs[-1], cs[-1])

## Input convex

In [ ]:
#| export
class FICNN(nn.Module):
    "Fully input-convex NN. Refer to [ICNN](https://arxiv.org/abs/1609.07152)."
    def __init__(self, ni: int,              # input size
                 nh: Union[int, list[int]],  # hidden size
                 no: int,                    # output size
                 num_layer: int=2,           # number of layers (include the output layer), if `nh` is a list, then the `num_layer = len(nh) + 1`
                 actn: str='relu',           # hidden activation
                 out_actn: str=None,         # output activation
                 init_gain: float=1.         # weight initialization gain
                ):
        super().__init__()
        nhs = [nh] * (num_layer - 1) if isinstance(nh, int) else nh
        self.w_y = nn.ModuleList([nn.Linear(ni, nh) for nh in nhs + [no]])
        self.w_z = nn.ModuleList([nn.Linear(i, j, bias=False) for i,j in zip(nhs, nhs[1:] + [no])])
        self.actn = getattr(F, actn)
        self.out_actn = getattr(F, out_actn) if out_actn is not None else noop

        lambda_init(self, lambda w,b: (nn.init.xavier_normal_(w, init_gain), nn.init.zeros_(b)))
        self.weight_constraint()
    
    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.w_z.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)
    
    def forward(self, x):
        z = self.w_y[0](x)
        if len(self.w_z) == 0: return self.out_actn(z)
        for wz,wy in zip(self.w_z[:-1], self.w_y[1:-1]):
            z = self.actn(wz(z) + wy(x))
        z = self.out_actn(self.w_z[-1](z) + self.w_y[-1](x))
        return z

In [ ]:
#| export
class ICRNN(nn.Module):
    """Input convex RNN. Refer to [Optimal Control Via Neural Networks: A Convex Approach](https://arxiv.org/abs/1805.11835).
    
    Inputs:  x, shape (N,L,D_in)
    Outputs: outputs, shape (N,L,D_out)
    """
    def __init__(self, ni: int,              # input size
                 nh: int,                    # hidden size
                 no: int,                    # output size
                 actn: str='relu',           # hidden activation
                 out_actn: str=None,         # output activation
                 expand_inp: bool=True,      # expand the input to [x, -x]
                 init_gain: float=1.         # weight initialization gain
                ):
        super().__init__()
        ni = ni * 2 if expand_inp else ni
        self.expand_inp,self.nh = expand_inp,nh
        self.actn = getattr(F, actn)
        self.out_actn = getattr(F, out_actn) if out_actn is not None else noop
        self.U = nn.Linear(ni, nh)
        self.V = nn.Linear(nh, no)
        self.W = nn.Linear(nh, nh)
        self.D1 = nn.Linear(nh, no)
        self.D2 = nn.Linear(ni, nh)
        self.D3 = nn.Linear(ni, no)

        lambda_init(self, lambda w,b: (nn.init.xavier_normal_(w, init_gain), nn.init.zeros_(b)))
        self.weight_constraint()

    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)

    def forward(self, x):
        if self.expand_inp: x = torch.cat([x, -x], dim=-1)
        x = torch.cat([torch.zeros(x.shape[0],1,x.shape[-1], device=x.device), x], dim=-2)
        z_curr = z_prev = torch.zeros(x.shape[0], self.nh, device=x.device)
        ys = []
        for xi,xj in zip(x[:,:-1,:].permute(1,0,2), x[:,1:,:].permute(1,0,2)):
            z_curr = self.actn(self.U(xj) + self.W(z_prev) + self.D2(xi))
            ys.append(self.out_actn(self.V(z_curr) + self.D1(z_prev) + self.D3(xj)))
            z_prev = z_curr
        return torch.stack(ys, 1)

In [ ]:
#| export
class ICLSTM(nn.Module):
    """Input convex LSTM. From [ICLSTM](https://arxiv.org/abs/2311.07202).
    A L-ICLSTM is a stack of [LSTM, Linear, ReLU] repeated L times. The output is mapped through another Linear layer.
    For it to be input convex, all weights are constrained to be non-negative, 
    and all activations are convex, non-decreasing and non-negative (no activation is applied to the output in the code.)
    """
    def __init__(self, ni, nh, no, num_layers=2, expand_inp=True, actn='relu', gate_actn='relu', **kwargs):
        super().__init__()
        ni = ni * 2 if expand_inp else ni
        self.expand_inp = expand_inp
        self.actn = getattr(F, actn, 'relu')
        self.blocks = nn.ModuleList([self._mk_block(ni, nh, actn, gate_actn, **kwargs) for _ in range(num_layers)])
        self.dense_out = nn.Linear(ni, no)
        # Nonnegative weights
        self.weight_constraint()

    def _mk_block(self, ni, nh, actn, gate_actn, **kwargs):
        m = nn.Sequential(LSTM(ni, nh, 1, actn=actn, gate_actn=gate_actn, **kwargs),
                          Select(), nn.Linear(nh, ni))
        default_init(m[2], normal=False)
        return m
        
    def weight_constraint(self):
        "Apply nonnegative weight constriant."
        with torch.no_grad():
            for n,p in self.named_parameters():
                if 'weight' in n:
                    p.clamp_min_(0)

    def forward(self, x):
        if self.expand_inp: x = torch.cat([x, -x], dim=-1)
        x0 = x.clone()
        for b in self.blocks:
            x = self.actn(b(x)) + x0
        return self.dense_out(x)

In [ ]:
#| export
class WeightConstraintCB(Callback):
    "Weight constraint callback."
    order = 10
    def after_step(self): self.learner.model.weight_constraint()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 61 column 1 (char 1196)